In [1]:
%load_ext autoreload

In [2]:
# autoreload chaque module spécifié par %aimport
%autoreload 1

%aimport src.kerasCallbacks
%aimport src.results

from IPython.display import display
from os import path
import time

import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.layers import Dense, Input, Dropout
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

from src import constants
from src.kerasCallbacks import BestWeightsRestorer
import src.results as resultsUtil
import src.runUtil as runUtil

In [3]:

def load_feature_set():

    file_path = path.join(constants.DATA_PATH, 'jmirderivatives_base_split.csv')
    
    ftrs = np.array(pd.read_csv(file_path, header=None).values[:,2:-1])
    lbls = np.array(pd.read_csv(file_path, header=None).values[:,-1])
    
    return ftrs, lbls

def scale_features(ftrs):
    
    scaler = StandardScaler()
    ftrs_scld = scaler.fit_transform(ftrs)
    
    return ftrs_scld

def one_hot_labels(lbls):
    
    lbls_1d = lbls.reshape(len(lbls), 1)
        
    oh_encoder = OneHotEncoder(sparse=False)
    lbls_oh = oh_encoder.fit_transform(lbls_1d)
    
    return lbls_oh

In [4]:
# Load the features and the labels
raw_ftrs, raw_lbls = load_feature_set()

lbls_oh = one_hot_labels(raw_lbls)
lbl_encoder = LabelEncoder()
lbl_encoded_lbls = lbl_encoder.fit_transform(raw_lbls)

ftrs = scale_features(raw_ftrs)

print(ftrs.shape)

(143644, 96)


In [5]:
def create_model(input_shape):
    
    inputs = Input(shape=input_shape, name='mlp_derivs_input')
    
    layer = Dense(90, activation='relu', name='mlp_derivs_dense_1')(inputs)
    layer = Dropout(0.025, name='mlp_derivs_dropout_1')(layer)
    layer = Dense(70, activation='relu', name='mlp_derivs_dense_2')(layer)
    layer = Dropout(0.025, name='mlp_derivs_dropout_2')(layer)
    layer = Dense(50, activation='relu', name='mlp_derivs_dense_3')(layer)
    layer = Dropout(0.025, name='mlp_derivs_dropout_3')(layer)
    layer = Dense(30, activation='relu', name='mlp_derivs_dense_4')(layer)
    
    outputs = Dense(25, activation='softmax', name='mlp_derivs_output')(layer)
    
    model = Model(inputs=inputs, outputs=outputs)
    
    model.compile(
        optimizer=Adam(), 
        loss='categorical_crossentropy', 
        metrics=[
            CategoricalAccuracy(name='accuracy')
        ]
    )
    
    return model

In [7]:
experiment_number = 0

In [8]:
def train_fold(x_train, y_train, x_val, y_val, run_name, fold=None):

    input_shape = x_train[0].shape
    model = create_model(input_shape)
    
    if fold is not None:
        run_name += '_fold' + str(fold)
        print('Training fold ' + str(fold))
    
    tk_board = TensorBoard(log_dir=constants.LOGS_PATH + run_name)
    weight_restorer = BestWeightsRestorer(monitor='val_loss', mode='min', verbose=1)
    
    history = model.fit(
        x=x_train,
        y=y_train,
        validation_data=[x_val, y_val],
        epochs=150,
        batch_size=2000,
        shuffle=True,
        callbacks=[tk_board, weight_restorer],
        verbose=1
    )
    
    prob_predictions = model.predict(x_val)
    predictions = prob_predictions.argmax(axis=1)
    
    return {
        'history': history.history,
        'predictions': predictions
    }

experiment_number += 1
run_name = 'new_base_mlp_deriv_cv_' + str(experiment_number)

results = resultsUtil.cross_validate(
    x=ftrs,
    y=lbls_oh,
    y_label_encoded=lbl_encoded_lbls,
    n_splits=5,
    train_func=train_fold,
    run_name=run_name
)

runUtil.save_run_results(run_name, results)

resultsdf = pd.DataFrame([results]).transpose()

display(resultsdf)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Training fold 1
Train on 114904 samples, validate on 28740 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/150
114904/114904 [==============================] - 1s 6us/sample - loss: 2.9987 - accuracy: 0.1263 - val_loss: 2.7840 - val_accuracy: 0.1639
Epoch 2/150
114904/114904 [==============================] - 0s 3us/sample - loss: 2.7227 - accuracy: 0.1773 - val_loss: 2.6410 - val_accuracy: 0.1938
Epoch 3/150
114904/114904 [==============================] - 0s 3us/sample - loss: 2.6228 - accuracy: 0.1999 - val_loss: 2.5727 - val_accuracy: 0.2099
Epoch 4/150
114904/114904 [==============================] - 0s 3us/sample - loss: 2.5719 - accuracy: 0.2131 - val_loss: 2.5377 - val_accuracy: 0.2214
Epoch 5/150
114904/114904 [==============================] - 0s 3us/sample - loss: 2.5425 - ac

Epoch 102/150
114904/114904 [==============================] - 0s 2us/sample - loss: 2.2776 - accuracy: 0.2959 - val_loss: 2.3504 - val_accuracy: 0.2769
Epoch 103/150
114904/114904 [==============================] - 0s 3us/sample - loss: 2.2764 - accuracy: 0.2949 - val_loss: 2.3497 - val_accuracy: 0.2760
Epoch 104/150
114904/114904 [==============================] - 0s 2us/sample - loss: 2.2772 - accuracy: 0.2950 - val_loss: 2.3514 - val_accuracy: 0.2751
Epoch 105/150
114904/114904 [==============================] - 0s 2us/sample - loss: 2.2756 - accuracy: 0.2949 - val_loss: 2.3503 - val_accuracy: 0.2768
Epoch 106/150
114904/114904 [==============================] - 0s 2us/sample - loss: 2.2744 - accuracy: 0.2952 - val_loss: 2.3532 - val_accuracy: 0.2746
Epoch 107/150
114904/114904 [==============================] - 0s 3us/sample - loss: 2.2742 - accuracy: 0.2944 - val_loss: 2.3484 - val_accuracy: 0.2773
Epoch 108/150
114904/114904 [==============================] - 0s 3us/sample - los

Epoch 5/150
114911/114911 [==============================] - 0s 3us/sample - loss: 2.5505 - accuracy: 0.2191 - val_loss: 2.5145 - val_accuracy: 0.2307
Epoch 6/150
114911/114911 [==============================] - 0s 3us/sample - loss: 2.5263 - accuracy: 0.2258 - val_loss: 2.4989 - val_accuracy: 0.2344
Epoch 7/150
114911/114911 [==============================] - 0s 3us/sample - loss: 2.5059 - accuracy: 0.2316 - val_loss: 2.4796 - val_accuracy: 0.2409
Epoch 8/150
114911/114911 [==============================] - 0s 2us/sample - loss: 2.4923 - accuracy: 0.2354 - val_loss: 2.4702 - val_accuracy: 0.2438
Epoch 9/150
114911/114911 [==============================] - 0s 2us/sample - loss: 2.4814 - accuracy: 0.2386 - val_loss: 2.4633 - val_accuracy: 0.2467
Epoch 10/150
114911/114911 [==============================] - 0s 3us/sample - loss: 2.4690 - accuracy: 0.2422 - val_loss: 2.4520 - val_accuracy: 0.2491
Epoch 11/150
114911/114911 [==============================] - 0s 3us/sample - loss: 2.4615 - 

Epoch 59/150
114911/114911 [==============================] - 0s 3us/sample - loss: 2.3211 - accuracy: 0.2809 - val_loss: 2.3528 - val_accuracy: 0.2751
Epoch 60/150
114911/114911 [==============================] - 0s 3us/sample - loss: 2.3200 - accuracy: 0.2816 - val_loss: 2.3537 - val_accuracy: 0.2766
Epoch 61/150
114911/114911 [==============================] - 0s 3us/sample - loss: 2.3190 - accuracy: 0.2829 - val_loss: 2.3532 - val_accuracy: 0.2743
Epoch 62/150
114911/114911 [==============================] - 0s 3us/sample - loss: 2.3180 - accuracy: 0.2828 - val_loss: 2.3520 - val_accuracy: 0.2759
Epoch 63/150
114911/114911 [==============================] - 0s 3us/sample - loss: 2.3173 - accuracy: 0.2825 - val_loss: 2.3531 - val_accuracy: 0.2737
Epoch 64/150
114911/114911 [==============================] - 0s 3us/sample - loss: 2.3148 - accuracy: 0.2850 - val_loss: 2.3509 - val_accuracy: 0.2757
Epoch 65/150
114911/114911 [==============================] - 0s 3us/sample - loss: 2.31

Epoch 113/150
114911/114911 [==============================] - 0s 3us/sample - loss: 2.2761 - accuracy: 0.2936 - val_loss: 2.3351 - val_accuracy: 0.2831
Epoch 114/150
114911/114911 [==============================] - 0s 2us/sample - loss: 2.2745 - accuracy: 0.2955 - val_loss: 2.3378 - val_accuracy: 0.2812
Epoch 115/150
114911/114911 [==============================] - 0s 2us/sample - loss: 2.2730 - accuracy: 0.2939 - val_loss: 2.3380 - val_accuracy: 0.2810
Epoch 116/150
114911/114911 [==============================] - 0s 3us/sample - loss: 2.2729 - accuracy: 0.2942 - val_loss: 2.3362 - val_accuracy: 0.2808
Epoch 117/150
114911/114911 [==============================] - 0s 3us/sample - loss: 2.2705 - accuracy: 0.2961 - val_loss: 2.3384 - val_accuracy: 0.2801
Epoch 118/150
114911/114911 [==============================] - 0s 3us/sample - loss: 2.2726 - accuracy: 0.2949 - val_loss: 2.3366 - val_accuracy: 0.2798
Epoch 119/150
114911/114911 [==============================] - 0s 3us/sample - los

...

114922/114922 [==============================] - 0s 3us/sample - loss: 2.3805 - accuracy: 0.2657 - val_loss: 2.3984 - val_accuracy: 0.2626
Epoch 28/150
114922/114922 [==============================] - 0s 3us/sample - loss: 2.3778 - accuracy: 0.2663 - val_loss: 2.3934 - val_accuracy: 0.2630
Epoch 29/150
114922/114922 [==============================] - 0s 3us/sample - loss: 2.3749 - accuracy: 0.2665 - val_loss: 2.3936 - val_accuracy: 0.2625
Epoch 30/150
114922/114922 [==============================] - 0s 3us/sample - loss: 2.3715 - accuracy: 0.2679 - val_loss: 2.3874 - val_accuracy: 0.2642
Epoch 31/150
114922/114922 [==============================] - 0s 3us/sample - loss: 2.3692 - accuracy: 0.2691 - val_loss: 2.3893 - val_accuracy: 0.2665
Epoch 32/150
114922/114922 [==============================] - 0s 3us/sample - loss: 2.3658 - accuracy: 0.2689 - val_loss: 2.3875 - val_accuracy: 0.2654
Epoch 33/150
114922/114922 [==============================] - 0s 3us/sample - loss: 2.3653 - accuracy

Epoch 81/150
114922/114922 [==============================] - 0s 3us/sample - loss: 2.2977 - accuracy: 0.2884 - val_loss: 2.3584 - val_accuracy: 0.2745
Epoch 82/150
114922/114922 [==============================] - 0s 3us/sample - loss: 2.2944 - accuracy: 0.2891 - val_loss: 2.3568 - val_accuracy: 0.2754
Epoch 83/150
114922/114922 [==============================] - 0s 2us/sample - loss: 2.2955 - accuracy: 0.2900 - val_loss: 2.3592 - val_accuracy: 0.2756
Epoch 84/150
114922/114922 [==============================] - 0s 3us/sample - loss: 2.2925 - accuracy: 0.2903 - val_loss: 2.3572 - val_accuracy: 0.2728
Epoch 85/150
114922/114922 [==============================] - 0s 3us/sample - loss: 2.2926 - accuracy: 0.2916 - val_loss: 2.3570 - val_accuracy: 0.2767
Epoch 86/150
114922/114922 [==============================] - 0s 2us/sample - loss: 2.2922 - accuracy: 0.2906 - val_loss: 2.3590 - val_accuracy: 0.2728
Epoch 87/150
114922/114922 [==============================] - 0s 3us/sample - loss: 2.28

114925/114925 [==============================] - 0s 3us/sample - loss: 2.3537 - accuracy: 0.2730 - val_loss: 2.3851 - val_accuracy: 0.2670
Epoch 38/150
114925/114925 [==============================] - 0s 3us/sample - loss: 2.3522 - accuracy: 0.2732 - val_loss: 2.3816 - val_accuracy: 0.2673
Epoch 39/150
114925/114925 [==============================] - 0s 3us/sample - loss: 2.3493 - accuracy: 0.2749 - val_loss: 2.3817 - val_accuracy: 0.2669
Epoch 40/150
114925/114925 [==============================] - 0s 3us/sample - loss: 2.3481 - accuracy: 0.2759 - val_loss: 2.3815 - val_accuracy: 0.2685
Epoch 41/150
114925/114925 [==============================] - 0s 3us/sample - loss: 2.3444 - accuracy: 0.2757 - val_loss: 2.3801 - val_accuracy: 0.2675
Epoch 42/150
114925/114925 [==============================] - 0s 3us/sample - loss: 2.3420 - accuracy: 0.2775 - val_loss: 2.3785 - val_accuracy: 0.2686
Epoch 43/150
114925/114925 [==============================] - 0s 3us/sample - loss: 2.3430 - accuracy

Epoch 91/150
114925/114925 [==============================] - 0s 3us/sample - loss: 2.2874 - accuracy: 0.2917 - val_loss: 2.3581 - val_accuracy: 0.2763
Epoch 92/150
114925/114925 [==============================] - 0s 3us/sample - loss: 2.2865 - accuracy: 0.2928 - val_loss: 2.3584 - val_accuracy: 0.2758
Epoch 93/150
114925/114925 [==============================] - 0s 3us/sample - loss: 2.2847 - accuracy: 0.2937 - val_loss: 2.3555 - val_accuracy: 0.2760
Epoch 94/150
114925/114925 [==============================] - 0s 3us/sample - loss: 2.2844 - accuracy: 0.2921 - val_loss: 2.3601 - val_accuracy: 0.2746
Epoch 95/150
114925/114925 [==============================] - 0s 3us/sample - loss: 2.2848 - accuracy: 0.2925 - val_loss: 2.3576 - val_accuracy: 0.2756
Epoch 96/150
114925/114925 [==============================] - 0s 3us/sample - loss: 2.2835 - accuracy: 0.2936 - val_loss: 2.3592 - val_accuracy: 0.2740
Epoch 97/150
114925/114925 [==============================] - 0s 3us/sample - loss: 2.28

,0
avg_val_f1_micro,0.279803
avg_val_f1_macro,0.262677
avg_val_accuracy,0.279803
avg_batch_val_accuracy,0.279803
avg_accuracy,0.299715
avg_val_loss,2.345093
avg_loss,2.258318
train_time,237.282015


In [6]:
def train_final_model(x, y, run_name):
    
    save_path = constants.MODELS_PATH + run_name + '.h5' 
    
    input_shape = x[0].shape
    model = create_model(input_shape)
    
    tk_board = TensorBoard(log_dir=constants.LOGS_PATH + run_name)
    
    model.fit(
        x=x,
        y=y,
        epochs=150,
        batch_size=2000,
        shuffle=True,
        callbacks=[tk_board],
        verbose=1
    )
    
    model.save(save_path)


train_final_model(ftrs, lbls_oh, 'base_mlp_deriv_final4')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Epoch 1/150
143644/143644 [==============================] - 1s 7us/sample - loss: 3.0152 - accuracy: 0.1189
Epoch 2/150
143644/143644 [==============================] - 0s 3us/sample - loss: 2.6988 - accuracy: 0.1831
Epoch 3/150
143644/143644 [==============================] - 1s 4us/sample - loss: 2.6008 - accuracy: 0.2070
Epoch 4/150
143644/143644 [==============================] - 0s 3us/sample - loss: 2.5539 - accuracy: 0.2193
Epoch 5/150
143644/143644 [==============================] - 0s 3us/sample - loss: 2.5242 - accuracy: 0.2272
Epoch 6/150
143644/143644 [==============================] - 1s 4us/sample - loss: 2.5043 - accuracy: 0.2329
Epoch 7/150
143644/143644 [==============================] - 1s 4us/sample - loss: 2.4892 - accuracy: 0.2369
Ep

143644/143644 [==============================] - 0s 3us/sample - loss: 2.3046 - accuracy: 0.2884
Epoch 67/150
143644/143644 [==============================] - 0s 3us/sample - loss: 2.3061 - accuracy: 0.2889
Epoch 68/150
143644/143644 [==============================] - 1s 3us/sample - loss: 2.3063 - accuracy: 0.2872
Epoch 69/150
143644/143644 [==============================] - 1s 4us/sample - loss: 2.3054 - accuracy: 0.2877
Epoch 70/150
143644/143644 [==============================] - 1s 3us/sample - loss: 2.3032 - accuracy: 0.2884
Epoch 71/150
143644/143644 [==============================] - 0s 3us/sample - loss: 2.3028 - accuracy: 0.2876
Epoch 72/150
143644/143644 [==============================] - 1s 3us/sample - loss: 2.2994 - accuracy: 0.2885
Epoch 73/150
143644/143644 [==============================] - 1s 4us/sample - loss: 2.3001 - accuracy: 0.2883
Epoch 74/150
143644/143644 [==============================] - 1s 4us/sample - loss: 2.2975 - accuracy: 0.2910
Epoch 75/150
143644/143

143644/143644 [==============================] - 0s 3us/sample - loss: 2.2611 - accuracy: 0.3011
Epoch 141/150
143644/143644 [==============================] - 0s 3us/sample - loss: 2.2618 - accuracy: 0.2996
Epoch 142/150
143644/143644 [==============================] - 0s 3us/sample - loss: 2.2614 - accuracy: 0.3007
Epoch 143/150
143644/143644 [==============================] - 0s 3us/sample - loss: 2.2594 - accuracy: 0.2999
Epoch 144/150
143644/143644 [==============================] - 0s 3us/sample - loss: 2.2574 - accuracy: 0.3004
Epoch 145/150
143644/143644 [==============================] - 0s 3us/sample - loss: 2.2582 - accuracy: 0.3006
Epoch 146/150
143644/143644 [==============================] - 0s 3us/sample - loss: 2.2586 - accuracy: 0.3002
Epoch 147/150
143644/143644 [==============================] - 0s 3us/sample - loss: 2.2567 - accuracy: 0.3008
Epoch 148/150
143644/143644 [==============================] - 0s 3us/sample - loss: 2.2560 - accuracy: 0.3016
Epoch 149/150
1